In [0]:
/* Databricks SQL Query: ADS_RDS_EVENT_STATUS_MEP */
/* Converted from notebook format for better performance */
/* Generated: 2025-09-23 09:33:15 */

/* Declare variables */ 
DECLARE OR REPLACE VARIABLE p_load_date STRING DEFAULT {{p_load_date}};
DECLARE OR REPLACE VARIABLE p_process_key STRING DEFAULT {{p_process_key}};
--DECLARE OR REPLACE VARIABLE map_id STRING DEFAULT 'ADS_RDS_EVENT_STATUS_MEP';
DECLARE OR REPLACE VARIABLE dif_table_name STRING DEFAULT 'gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_MEP_ADS_MAP_SCD_DIFF';
--DECLARE OR REPLACE VARIABLE max_key BIGINT DEFAULT 0;

-- Get the maximum EST_KEY from Target table
/*SET VAR max_key = (
    SELECT COALESCE(MAX(est_key), 0) 
    FROM gap_catalog.ads_owner.event_status
);*/

/* Truncate XC Table */
TRUNCATE TABLE gap_catalog.ads_etl_owner.XC_RDS_EVENT_STATUS_MEP;

/* Fill XC Table */
INSERT INTO gap_catalog.ads_etl_owner.XC_RDS_EVENT_STATUS_MEP 
      with C_0FILTERAR0J94H5CGV4DIAG7PJNCDAKGK as (
      select
          RDS_MEPOPERATIONSTATE.id  ID,
          RDS_MEPOPERATIONSTATE.value  VALUE
        from gap_catalog.ads_etl_owner.DLK_ADS_LOV_RDS_MEPOPERATIONSTATE RDS_MEPOPERATIONSTATE
        where ( CAST(from_utc_timestamp(SYS_EFFECTIVE_DATE, 'Europe/Prague') AS DATE) = CAST(p_load_date AS DATE)
            and RDS_MEPOPERATIONSTATE.sys = 'Brasil'
            and RDS_MEPOPERATIONSTATE.lang = 'CZ'
              )
      )
      select  /*+no hint*/
              /*seznam vkladanych nebo updatovanych sloupcu bez SK, auditnich atribudu a deleted flagu*/
          FILTER_A.ID EST_SOURCE_ID,
          'RDS' EST_SOURCE_SYSTEM_ID,
          'RDS_MEPOPERATIONSTATE' EST_SOURCE_SYS_ORIGIN,
          FILTER_A.VALUE EST_DESC,
          'N' EST_DELETED_FLAG
        from (C_0FILTERAR0J94H5CGV4DIAG7PJNCDAKGK FILTER_A)
        where (1=1)
;

/* Cleanup DIFF Table */
DROP TABLE IF EXISTS gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_MEP_ADS_MAP_SCD_DIFF;

/* Create DIFF Table */
create  table gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_MEP_ADS_MAP_SCD_DIFF
( tech_del_flg  char(1),
  tech_new_rec  char(1),
  tech_rid      varchar(255),
  EST_KEY  INTEGER,
  EST_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
  EST_SOURCE_ID  varchar(120),
  EST_SOURCE_SYSTEM_ID  varchar(120),
  EST_SOURCE_SYS_ORIGIN  varchar(120),
  EST_DESC  varchar(400))
;

/* Populate DIFF Table - New/Updated Records */
insert into IDENTIFIER(dif_table_name)
(
  tech_del_flg,
  tech_new_rec,
  tech_rid,
  EST_KEY,
  EST_SOURCE_ID,
  EST_SOURCE_SYSTEM_ID,
  EST_SOURCE_SYS_ORIGIN,
  EST_DESC
)
select   'N' as tech_del_flg, 
    case when trg.EST_SOURCE_ID is null then 'Y' else 'N' end as tech_new_rec,
    trg.rid as tech_rid,
   trg.EST_KEY, 
   src.EST_SOURCE_ID, 
   src.EST_SOURCE_SYSTEM_ID, 
   src.EST_SOURCE_SYS_ORIGIN, 
   src.EST_DESC
 from 
    (select /*+ full(xc) */      EST_SOURCE_ID, 
       EST_SOURCE_SYSTEM_ID, 
       EST_SOURCE_SYS_ORIGIN, 
       EST_DESC, 
       EST_DELETED_FLAG
       from gap_catalog.ads_etl_owner.XC_RDS_EVENT_STATUS_MEP xc
    where (EST_SOURCE_SYS_ORIGIN='RDS_MEPOPERATIONSTATE')) src LEFT JOIN
    (select  est_key||'.'||est_valid_from as rid, t.* from gap_catalog.ads_owner.EVENT_STATUS t
      where EST_CURRENT_FLAG  = 'Y'
        and EST_VALID_TO  = to_date('01013000','ddMMyyyy')
       and (EST_SOURCE_SYS_ORIGIN='RDS_MEPOPERATIONSTATE')      ) trg
ON trg.EST_SOURCE_ID = src.EST_SOURCE_ID
 and trg.EST_SOURCE_SYSTEM_ID = src.EST_SOURCE_SYSTEM_ID
 and trg.EST_SOURCE_SYS_ORIGIN = src.EST_SOURCE_SYS_ORIGIN
 and trg.EST_VALID_TO = to_date('30000101','yyyyMMdd') WHERE (
     decode( src.EST_DESC,trg.EST_DESC,1,0 ) = 0  or
     decode( src.EST_DELETED_FLAG,trg.EST_DELETED_FLAG,1,0 ) = 0 or 
     trg.EST_SOURCE_ID is null   );

/* Populate DIFF Table - Deleted Records */
insert into IDENTIFIER(dif_table_name)
(
  tech_del_flg,
  tech_new_rec,
  tech_rid,
  EST_KEY,
  EST_SOURCE_ID,
  EST_SOURCE_SYSTEM_ID,
  EST_SOURCE_SYS_ORIGIN,
  EST_DESC
)
select   'Y' as tech_del_flg, 
 'N' as tech_new_rec, 
 trg.rid as tech_rid, 
   trg.EST_KEY, 
   trg.EST_SOURCE_ID, 
   trg.EST_SOURCE_SYSTEM_ID, 
   trg.EST_SOURCE_SYS_ORIGIN, 
   trg.EST_DESC
 from 
    (select  est_key||'.'||est_valid_from as rid, t.* from gap_catalog.ads_owner.EVENT_STATUS t
      where EST_CURRENT_FLAG  = 'Y' and EST_DELETED_FLAG  = 'N'
        and EST_VALID_TO  = to_date('01013000','ddMMyyyy')
       and (EST_SOURCE_SYS_ORIGIN='RDS_MEPOPERATIONSTATE')      ) trg LEFT JOIN
    (select /*+ full(xc) */      EST_SOURCE_ID, 
       EST_SOURCE_SYSTEM_ID, 
       EST_SOURCE_SYS_ORIGIN, 
       to_date('30000101','yyyyMMdd') as EST_VALID_TO
       from gap_catalog.ads_etl_owner.XC_RDS_EVENT_STATUS_MEP xc
    where (EST_SOURCE_SYS_ORIGIN='RDS_MEPOPERATIONSTATE')) src
ON trg.EST_SOURCE_ID = src.EST_SOURCE_ID
 and trg.EST_SOURCE_SYSTEM_ID = src.EST_SOURCE_SYSTEM_ID
 and trg.EST_SOURCE_SYS_ORIGIN = src.EST_SOURCE_SYS_ORIGIN
 and trg.EST_VALID_TO = src.EST_VALID_TO WHERE (src.EST_SOURCE_ID is null);

/* Close Old Records in Target */
update  gap_catalog.ads_owner.EVENT_STATUS set
       EST_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
       EST_UPDATE_PROCESS_KEY = CAST(p_process_key AS BIGINT),
       EST_CURRENT_FLAG = 'N', 
       EST_VALID_TO = to_date(CAST(p_load_date AS DATE),'yyyy-MM-dd')-1
 where EST_CURRENT_FLAG = 'Y'
   and EST_VALID_TO = to_date('30000101','yyyyMMdd')
   and est_key||'.'||est_valid_from in (select tech_rid from IDENTIFIER(dif_table_name) where tech_rid is not null);

/* Insert Changed Records */
insert  into gap_catalog.ads_owner.EVENT_STATUS 
 ( EST_KEY, 
   EST_SOURCE_ID, 
   EST_SOURCE_SYSTEM_ID, 
   EST_SOURCE_SYS_ORIGIN, 
   EST_DESC, 
   EST_VALID_FROM, 
   EST_VALID_TO, 
   EST_CURRENT_FLAG, 
   EST_DELETED_FLAG, 
   EST_INSERTED_DATETIME, 
   EST_INSERT_PROCESS_KEY, 
   EST_UPDATED_DATETIME, 
   EST_UPDATE_PROCESS_KEY)
select EST_KEY, 
   EST_SOURCE_ID, 
   EST_SOURCE_SYSTEM_ID, 
   EST_SOURCE_SYS_ORIGIN, 
   EST_DESC, 
   to_date(CAST(p_load_date AS DATE),'yyyy-MM-dd') as EST_VALID_FROM, 
   DATE('3000-01-01') as EST_VALID_TO, 
   'Y' as EST_CURRENT_FLAG, 
   tech_del_flg as EST_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as EST_INSERTED_DATETIME, 
   CAST(p_process_key AS BIGINT) as EST_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as EST_UPDATED_DATETIME, 
   CAST(p_process_key AS BIGINT) as EST_UPDATE_PROCESS_KEY
  from IDENTIFIER(dif_table_name)
 where tech_new_rec = 'N';

/* Insert New Records */
insert  into gap_catalog.ads_owner.EVENT_STATUS 
 ( EST_KEY, 
   EST_SOURCE_ID, 
   EST_SOURCE_SYSTEM_ID, 
   EST_SOURCE_SYS_ORIGIN, 
   EST_DESC, 
   EST_VALID_FROM, 
   EST_VALID_TO, 
   EST_CURRENT_FLAG, 
   EST_DELETED_FLAG, 
   EST_INSERTED_DATETIME, 
   EST_INSERT_PROCESS_KEY, 
   EST_UPDATED_DATETIME, 
   EST_UPDATE_PROCESS_KEY)
select 
   --EST_KEY_NEW + max_key AS EST_KEY,  -- Add the offset to maintain key sequence, 
   xxhash64(uuid()) AS EST_KEY,
   EST_SOURCE_ID, 
   EST_SOURCE_SYSTEM_ID, 
   EST_SOURCE_SYS_ORIGIN, 
   EST_DESC, 
   to_date(CAST(p_load_date AS DATE),'yyyy-MM-dd') as EST_VALID_FROM, 
   DATE('3000-01-01') as EST_VALID_TO, 
   'Y' as EST_CURRENT_FLAG, 
   tech_del_flg as EST_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as EST_INSERTED_DATETIME, 
   CAST(p_process_key AS BIGINT) as EST_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as EST_UPDATED_DATETIME, 
   CAST(p_process_key AS BIGINT) as EST_UPDATE_PROCESS_KEY
  from IDENTIFIER(dif_table_name)
 where tech_new_rec = 'Y';